# 1.特約藥局搜尋地址、經緯度
#### 前處理

In [1]:
import numpy as pd
import pandas as pd

url = 'https://data.nhi.gov.tw/Datasets/Download.ashx?rid=A21030000I-D50001-001&l=https://data.nhi.gov.tw/resource/mask/maskdata.csv'
df = pd.read_csv(url, encoding='utf8')
print(df.columns)
df.head()

Index(['醫事機構代碼', '醫事機構名稱', '醫事機構地址', '醫事機構電話', '成人口罩剩餘數', '兒童口罩剩餘數', '來源資料時間'], dtype='object')


,醫事機構代碼,醫事機構名稱,醫事機構地址,醫事機構電話,成人口罩剩餘數,兒童口罩剩餘數,來源資料時間
0,0145080011,衛生福利部花蓮醫院豐濱原住民分院,花蓮縣豐濱鄉豐濱村光豐路４１號,(03)8358141,516,291,2020/02/14 22:32:05
1,0291010010,連江縣立醫院,連江縣南竿鄉復興村２１７號,(083)623995,0,80,2020/02/14 22:32:05
2,2331240014,新北市平溪區衛生所,新北市平溪區公園街１７號１樓,(02)24951015,0,284,2020/02/14 22:32:05
3,2331260016,新北市貢寮區衛生所,新北市貢寮區朝陽街７０巷１０號、新北市貢寮區仁愛路１２８號,(02)24901431,0,152,2020/02/14 22:32:05
4,2331290014,新北市烏來區衛生所,新北市烏來區新烏路５段１０９號,(02)26617200,0,24,2020/02/14 22:32:05


In [2]:
# # 檢查遺失值
# df[df['醫事機構地址'].isna()].sum()
addr = input('請輸入部分地址：')
df2 = df[df['醫事機構地址'].str.contains(addr)]
print('df2.shape =', df2.shape)

請輸入部分地址：台南市北區
df2.shape = (23, 7)


In [3]:
# geocoder地圖服務：找經緯度、郵遞區號等
    # 安裝：conda install -c conda-forge geocoder
import geocoder
# 第1筆資料地址為例
for addr in df2.iloc[:1, 2]:
    print(addr)
    g = geocoder.arcgis(addr)
    # 經緯度
    latitude = g.latlng[0]   # 緯度
    longitude = g.latlng[1]  # 經度
    print('經緯度 =', g.latlng)
    # 轉成json 詳細資料
    print(g.json)
    print('郵遞區號 =', g.json['address'].split(',')[1].strip())

台南市北區延平里公園路５４６號
經緯度 = [23.00692285206878, 120.21006925812218]
{'address': '台南市北區國姓里公園路546號, 704', 'bbox': {'northeast': [23.007934000000002, 120.21111099999999], 'southwest': [23.005934, 120.20911099999998]}, 'confidence': 9, 'lat': 23.00692285206878, 'lng': 120.21006925812218, 'ok': True, 'quality': 'PointAddress', 'raw': {'name': '台南市北區國姓里公園路546號, 704', 'extent': {'xmin': 120.20911099999998, 'ymin': 23.005934, 'xmax': 120.21111099999999, 'ymax': 23.007934000000002}, 'feature': {'geometry': {'x': 120.21006925812218, 'y': 23.00692285206878}, 'attributes': {'Score': 97.94, 'Addr_Type': 'PointAddress'}}}, 'score': 97.94, 'status': 'OK'}
郵遞區號 = 704


In [4]:
# 地圖中心點定義
center_latitude = geocoder.arcgis('台南市北區').latlng[0]
center_longitude = geocoder.arcgis('台南市北區').latlng[1]
# 加入經緯度
df2['緯度'] = [geocoder.arcgis(i).latlng[0] for i in df2['醫事機構地址']]
df2['經度'] = [geocoder.arcgis(i).latlng[1] for i in df2['醫事機構地址']]
df2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,醫事機構代碼,醫事機構名稱,醫事機構地址,醫事機構電話,成人口罩剩餘數,兒童口罩剩餘數,來源資料時間,緯度,經度
2178,5921040169,信心藥局,台南市北區延平里公園路５４６號,(06)2822209,0,172,2020/02/14 22:32:05,23.006923,120.210069
2179,5921040212,真安藥局,台南市北區公園路６９８號,(06)2819836,2,174,2020/02/14 22:32:05,23.008510,120.210128
2180,5921040221,好鄰居藥局,台南市北區西華里公園路１２８號２樓７室,(06)2219015,0,164,2020/02/14 22:32:05,22.998898,120.207524
2181,5921040276,全一藥局,台南市北區大豐里和緯路四段９５巷３８號,(06)2580021,0,128,2020/02/14 22:32:05,23.007575,120.191333
2182,5921040294,澄清堂藥局,台南市北區勝利路４９５號,(06)2377224,0,238,2020/02/14 22:32:05,23.006455,120.218532
2183,5921040301,永生藥局,台南市北區六甲里公園路４８１號,(06)2813681,14,196,2020/02/14 22:32:05,23.008937,120.210211
2184,5921040392,宏吉藥局,台南市北區大港里大興街２２４號,(06)2589692,0,26,2020/02/14 22:32:05,23.010837,120.187877
2185,5921040409,華成藥局,台南市北區華興里南園街１１１號,(06)2344553,0,78,2020/02/14 22:32:05,23.007571,120.225726
2186,5921040436,嘉洲藥局,台南市北區文元里文賢路２４０號,(06)2583180,0,84,2020/02/14 22:32:05,23.004241,120.193761
2187,5921040481,眾安藥局,台南市北區文元里文賢路１０３４號,(06)2588339,0,152,2020/02/14 22:32:05,23.014161,120.193829


#### 地圖

In [5]:
import folium
# 創建地圖
m = folium.Map(location=[center_latitude, center_longitude], zoom_start=15)
# 標示資料點
from folium import plugins
incidents = plugins.MarkerCluster().add_to(m)
# 將資料點加到地圖上
for lat, lng, name, leftAd, leftCh in zip([j for j in df2['緯度']], [j for j in df2['經度']], 
                                          [j for j in df2['醫事機構名稱']], 
                                          [j for j in df2['成人口罩剩餘數']], [j for j in df2['兒童口罩剩餘數']]):
    folium.Circle(location=[lat, lng], radius=20, color="yellow", weight=3, 
                  fill_color="red", fill_opacity=0.8).add_to(m)
    folium.Marker(location=[lat, lng], icon=None, popup=name+'成人'+str(leftAd)+'兒童'+str(leftCh)).add_to(incidents)

# # 顯示查詢地圖：實作時記得下一行取消註解
# m

# # 查詢結果存成html
# m.save("map.html")

### 大家可自行跑程式碼查詢，由於是動態網頁故無法顯示在notebook當中
#### [查詢出來內容範例圖-請點我連到網頁(3.6Mb 大家可等一下^^")](https://github.com/ordersam/pythoncase/blob/master/1.%E5%8F%A3%E7%BD%A9%E6%9F%A5%E8%A9%A2-result.png)
    # 可以移動、縮放、點選